# General data questions and exploration


In [1]:
##TODO:
# Try ML with and without network metrics
# Test at different time periods
# Test other event datasets
# Check steps from book - change to best practice sklearn


# What correlates with witness label
# GPS accounts tend to be spam/businesses?
# Compare GPS from stream / profile / hand coding
# Explore age of account
# Is detecting co-occuring tags viable?
# What kind of data/user is likely to be deleted?
# Is user name change / user deletion/protection a useful predictor
# Compare Change in network, whether it's useful to collect.
# Check gps count, location in profile
# Check timezone distribution
# 'Ordinary person' vs bot/celeb/business/news -- using source field, tweet rate, timezone

# prop of gps -- users and tweets. Automated, instagram sourced?
# prop of sources
# prop of media/urls
# users with location on profile? Some set 'in solidarity'?
# Cycadian posting rythym - can identify real people vs bots?
# location via friend network?
# language

# \item Tweets which were automatically generated from Instagram posts were much more likely to include GPS coordinates, and as media, more likely to represent a ground truth. Therefore this content may be worth focusing on.
# \item Aid requests were very rare. Those that were identified were often reposts rather than originals, and are often referring to the same original message which begins to trend.
# \item Info for affected class should differentiate between immediate and non-immediate content. E.g. a call to mobilise a clean-up or rescue crew vs. a link to an insurance claim form.
# \item For `unrelated' messages, those which matched the keyword stream were highly represented by automated messages coming from a particular set of sources which presumably uses trending tags to gain exposure. This is easy to pre-filter.
# \item Geographically-tagged Tweets are predominantly either: Instagram cross-posts, or automatically generated job listings from a small set of sources (and therefore easy to pre-filter).
        

# Sum of network edge reciprocity
# k-cohesiveness -- Structural cohesion

In [43]:
### Initialisation ###
import os
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4]

EVENT_NAME = Event.objects.all()[0].name.replace(' ', '')
DIR = './data/harvey_user_location/'
DF_FILENAME = 'df_users.csv'
LATEX_DAT_FILE = 'harvey-exploration-data.dat'
#LATEX_TABLE_FILE = 'harvey-exploration-tabs.dat'

# Confirm correct database is set in Django settings.py
if 'Harvey' not in EVENT_NAME:
    raise Exception('Event name mismatch -- check database set in Django')

# Open original Dataframe
users_df = pd.read_csv(DIR + DF_FILENAME, index_col=0)
users_df.shape

(1500, 46)

In [9]:
def set_tex_var(name, val, dec=3):
    """
    Define a value as a latex variable to be
    imported into Latex file with (e.g.):
    \input{data/harvey-coding.dat}

    NOTE: does not look for duplicate variable definitions.
    """
    if isinstance(val, float):
        template = r'\newcommand{{\{}}}{{{:,.' + str(dec) + r'f}}}'
    elif isinstance(val, int):
        template = r'\newcommand{{\{}}}{{{:,}}}'
    else:
        template = r'\newcommand{{\{}}}{{{}}}'
    with open(LATEX_DAT_FILE, 'a') as a_writer:
        a_writer.write(template.format(name, val) + '\n')

First, given we are treating the data as binary-coded, we need to be aware of the third code of 'unsure'. In most comparisons, these will automatically be considered as the opposite code from what is being tested. We can instead manually set it to either value, or remove it entirely, given that it comprises a small proportion of the data.

In [3]:
unsure_code = (users_df.is_coded_as_witness == 0) & (users_df.is_coded_as_non_witness == 0)
print(sum(unsure_code), 'cases coded as \'unsure\'')

# Remove 'unsure' rows from data:
#users_df = users_df.loc[unsure_code==False]

# Assign 'unsure' rows to positive coded case:
#users_df.is_coded_as_witness = (users_df.is_coded_as_non_witness == False).astype(int)

# Assign 'unsure' rows to negative coded case:
#users_df.is_coded_as_non_witness = (users_df.is_coded_as_witness == False).astype(int)

31 cases coded as 'unsure'


## Geographic Metadata and Manual Coding
Manual coding of users targetted the perceived locality of the user to the event. We can compare the geographic metadata provided by Twitter to these codes to determine their usefulness as a predictor for this value.

### Profile Location Field
The first value to check is the location of a user as set in their profile. This is a user-set string. In an earlier notebook, this string was geocoded using Google Maps api and evaluated for whether it fell within the bounding box defined for this event. We can therefore check whether this test correlates with the coded value.

First, we check what proportion of users provide a value in the field. We can then generate a confusion matrix showing the agreement between the profile locality where provided, and the coded value.

In [4]:
us = User.objects.filter(user_class__gt=0)
#us = us.filter(coding_for_user__coding_id=1, coding_for_user__data_code__data_code_id__gt=0)
tot = us.count()
tot_loc = us.count() - (us.filter(location="") | us.filter(location__isnull=True)).count()

print('Total users: ', tot)
print('Total users with location filled: ', tot_loc)
print('Proportion: {:.4}%'.format((tot_loc/tot)*100))

print('\nProportion of coded users with location filled: {:.4}%'.format((sum(users_df.location.notna())/users_df.shape[0])*100))

print('\nProportion of coded users with parseable location filled: {:.4}%'.format(
    (users_df.loc[(users_df.is_non_local_profile_location + users_df.is_local_profile_location) > 0].shape[0]/users_df.shape[0])*100))


Total users:  31932
Total users with location filled:  25619
Proportion: 80.23%

Proportion of coded users with location filled: 79.4%

Proportion of coded users with parseable location filled: 77.47%


In [5]:
vals = users_df.loc[users_df["is_local_profile_location"] == 1]["is_coded_as_witness"].value_counts()
vals2 = users_df.loc[users_df["is_coded_as_witness"] == 1]["is_local_profile_location"].value_counts()

print('{} of {} ({:.4}%) users were classified as having a local profile'.format(sum(vals), len(users_df), sum(vals)/len(users_df)*100))
print('{} of {} ({:.4}%) users were coded as a witness'.format(sum(vals2), len(users_df), sum(vals2)/len(users_df)*100))
print('{} of {} ({:.4}%) users with local profile locations were coded as witness'.format(vals[1], sum(vals), vals[1]/sum(vals)*100))
print('{} of {} ({:.4}%) witness codes had a local profile'.format(vals2[1], sum(vals2), vals2[1]/sum(vals2)*100))

397 of 1500 (26.47%) users were classified as having a local profile
386 of 1500 (25.73%) users were coded as a witness
258 of 397 (64.99%) users with local profile locations were coded as witness
258 of 386 (66.84%) witness codes had a local profile


In [9]:
import pandas as pd
import math
import numpy as np
#from sklearn.metrics import matthews_corrcoef


def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )


def confusion_matrix_from_series(s1, s2):
    """
    Returns confusion matrix for two binary
    series
    """
    df = pd.concat([s1, s2], axis=1)
    return confusion_matrix(df, s1.name, s2.name)


def calc_agreement_coefs(df: pd.DataFrame):
    """
    Calculates Cohen's Kappa and
    Krippendorff's Alpha for a
    given confusion matrix.
    """
    arr = df.to_numpy()
    n = arr.sum()
    p_o = 0
    for i in range(len(arr)):
        p_o += arr[i][i]/n
    p_e = 0
    for i in range(len(arr)):
        p_e += (arr.sum(axis=1)[i] *
                arr.sum(axis=0)[i]) / (n*n)
    kappa = (p_o-p_e)/(1-p_e)
    
    phi_num = (arr[0][0]*arr[1][1]) - (arr[0][1]*arr[1][0])
    a = np.sum(conf, axis=1)
    b = np.sum(conf, axis=0)
    phi_denom = math.sqrt(np.prod([a, b]))
    phi = phi_num/phi_denom
    phi_max = min(
        math.sqrt(a[0] * b[1])/math.sqrt(a[1] * b[0]),
        math.sqrt(a[1] * b[0])/math.sqrt(a[0] * b[1])
        )
    phiphimax = phi/phi_max
    
    coin_arr = np.transpose(arr) + arr
    exp_distribution = [sum(x) for x in coin_arr]
    p_e_krippendorf = sum([a * (a-1) for a in exp_distribution])/(2*n*((2*n)-1))
    alpha = (p_o - p_e_krippendorf) / (1-p_e_krippendorf)
    
    return p_o, kappa, alpha, phi, phiphimax


def calc_agreement_metrics(df: pd.DataFrame):
    """
    Calculates various agreement metrics
    for a given binary confusion matrix.
    
    Assumes true condition as ROW heading and
    ascending integer labels.
    """
    arr = df.to_numpy()
    if len(arr) != 2:
        print('ERROR: df is not 2x2')
        return None
    results = {}
    results['Prevalence'] = arr.sum(axis=0)[1]/arr.sum()
    results['Accuracy'] = (arr[0][0] + arr[1][1])/arr.sum()
    results['Prec'] = arr[1][1]/arr.sum(axis=1)[1]
    results['Recall'] = arr[1][1]/arr.sum(axis=0)[1]
    results['f1Score'] = (2 * results['Prec'] * results['Recall'])/(results['Prec']+results['Recall'])
    results['Specificity'] = arr[0][0]/arr.sum(axis=0)[0]
    results['FalseNegRate'] = arr[0][1]/arr.sum(axis=0)[1]
    p_o, kappa, alpha, phi, phiphimax = calc_agreement_coefs(df)
    results['Cohen\'s Kappa'] = kappa
    results['Krippendorff\'s Alpha'] = alpha
    results['Phi Coef / MCC'] = phi
    results['Phi/PhiMax'] = phiphimax
    
    return results
    

In [10]:
# We exclude rows where either no profile location field was provided, 
# or the location was not parsed by the API:
loc_df = users_df.loc[(users_df.is_non_local_profile_location + users_df.is_local_profile_location) > 0]
print(users_df.shape[0] - loc_df.shape[0], 'rows with no parseable profile location value excluded')

conf = confusion_matrix(loc_df, 'is_local_profile_location', 'is_coded_as_witness')
conf

338 rows with no parseable profile location value excluded


is_coded_as_witness,0,1
is_local_profile_location,,
0,695,70
1,139,258


In [11]:
results = calc_agreement_metrics(conf)
res_df = pd.DataFrame.from_dict(results, orient='index', columns=['loc_prof_nona'])
res_df

,loc_prof_nona
Prevalence,0.282272
Accuracy,0.820138
Prec,0.649874
Recall,0.786585
f1Score,0.711724
Specificity,0.833333
FalseNegRate,0.213415
Cohen's Kappa,0.582731
Krippendorff's Alpha,0.581198
Phi Coef / MCC,0.588340


As excluding the ~20% of values with no parseable location field provided is not an option in practice, we must decide to either discard them (i.e. by default classify as non-local) or include them (default classify as local). The first option will inevitable discard true positive cases, thus reducing recall, whereas the latter will introduce false positives, reducing precision:

In [12]:
conf = confusion_matrix(users_df, 'is_local_profile_location', 'is_coded_as_witness')
print('loc_prof_notna')
print(conf)
res_df['loc_prof_notna'] = calc_agreement_metrics(conf).values()


conf = confusion_matrix_from_series(
    pd.Series(
                (users_df.is_local_profile_location) | (users_df.location.isna()).astype(int), 
                name='is_local_profile_location_or_na'
             ),
    users_df.is_coded_as_witness)
print('\nloc_prof_orna')
print(conf)
res_df['loc_prof_orna'] = calc_agreement_metrics(conf).values()


res_df

loc_prof_notna
is_coded_as_witness          0    1
is_local_profile_location          
0                          975  128
1                          139  258

loc_prof_orna
is_coded_as_witness                0    1
is_local_profile_location_or_na          
0                                714   80
1                                400  306


,loc_prof_nona,loc_prof_notna,loc_prof_orna
Prevalence,0.282272,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000
Prec,0.649874,0.649874,0.433428
Recall,0.786585,0.668394,0.792746
f1Score,0.711724,0.659004,0.560440
Specificity,0.833333,0.875224,0.640934
FalseNegRate,0.213415,0.331606,0.207254
Cohen's Kappa,0.582731,0.538603,0.341243
Krippendorff's Alpha,0.581198,0.538725,0.309098
Phi Coef / MCC,0.588340,0.538700,0.379833


The results above are as expected. Excluding empty fields gives a precision/recall of 0.650/0.668 whereas exluding them gives 0.433/0.793. The exclusion strategy provides the highest f1 score, however given the purpose of the algorithm must be considered when choosing how to weight precision and recall metrics. For example, given the algorithm is designed to curate the feed for human consumption, a high precision is only necessary if the rate of positive cases exceeds the humans' ability to parse the incoming stream. Where the rate is low, sacrificing precision is acceptable to present the human user with more cases which they can then manually filter. 

This concept will be explored in more depth later in the project. For now, it is sufficient to note the values as a baseline model.

### Tweet Stream Coordinates
When posting a Tweet, a user may attach geographic coordinates. The location of the device is provided by the hardware and automatically included with the Tweet (thus the user does not influence the input). A Tweet may also include, instead of specific coordinates, a 'Place' object -- a geographic region (defined by Twitter) which typically describes a location such as a city, state or other similarly-sized region.

To geolocate a user, we can therefore investigate their Twitter feed for any Tweets containing this geographic data and compare these to the bounding box of the observed event. The derived field therefore represents whether *any* of a user's Tweets were identified as 'local' during the event.

For this dataset, the Twitter feed for each observed user spanning the duration of the collection period was collected at the end of the collection period. The feed is therefore made up of Tweets detected during the collection period, and any other Tweets the user made during the period, before or after the detected Tweet, provided they existed at the end of the collection period.

Further work on this area should consider the following:
* Where a local Tweet has been detected, check the proportion of other Tweets containing geographic data.
* Where other geo-Tweets exist, check whether they are from the same point or move around -- consider recoding for where a user Tweets from within *and* without the bounding box.
* Where all geo-Tweets come from the same point, it is likely that location has been manually set (i.e. it is a storefront/business account). This may be verifiable by checking the Tweet source.

In [13]:
res_df['local_tw'] = calc_agreement_metrics(conf).values()
res_df

,loc_prof_nona,loc_prof_notna,loc_prof_orna,local_tw
Prevalence,0.282272,0.257333,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000,0.680000
Prec,0.649874,0.649874,0.433428,0.433428
Recall,0.786585,0.668394,0.792746,0.792746
f1Score,0.711724,0.659004,0.560440,0.560440
Specificity,0.833333,0.875224,0.640934,0.640934
FalseNegRate,0.213415,0.331606,0.207254,0.207254
Cohen's Kappa,0.582731,0.538603,0.341243,0.341243
Krippendorff's Alpha,0.581198,0.538725,0.309098,0.309098
Phi Coef / MCC,0.588340,0.538700,0.379833,0.379833


While the recall of this metric is close (slightly less than) `loc_prof_notna`, there is no increase in precision and thus it is an inferior metric to predict the true condition.

We can however check whether the metric is capturing a different proportion of local users, and therefore improve upon the existing measures through combination. Using an OR condition will increase recall at the cost of precision; using an AND condition will increase precision at the cost of recall.

In [14]:
conf = confusion_matrix(users_df, 'has_tweet_from_locality', 'is_coded_as_witness')
conf

is_coded_as_witness,0,1
has_tweet_from_locality,,
0,898,135
1,216,251


In [15]:
conf = confusion_matrix_from_series(
    pd.Series(
        ((users_df.is_local_profile_location) | (users_df.has_tweet_from_locality)),
        name = 'is_local_profile_location_or_local_tw'
        ),
    users_df.is_coded_as_witness)
# conf

res_df['loc_prof_notna_or_loc_tw'] = calc_agreement_metrics(conf).values()

conf = confusion_matrix_from_series(
    pd.Series(
        ((users_df.is_local_profile_location) & (users_df.has_tweet_from_locality)),
        name = 'is_local_profile_location_and_local_tw'
        ),
    users_df.is_coded_as_witness)
# conf

res_df['loc_prof_notna_and_loc_tw'] = calc_agreement_metrics(conf).values()

res_df

,loc_prof_nona,loc_prof_notna,loc_prof_orna,local_tw,loc_prof_notna_or_loc_tw,loc_prof_notna_and_loc_tw
Prevalence,0.282272,0.257333,0.257333,0.257333,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000,0.680000,0.788000,0.800000
Prec,0.649874,0.649874,0.433428,0.433428,0.558219,0.653571
Recall,0.786585,0.668394,0.792746,0.792746,0.844560,0.474093
f1Score,0.711724,0.659004,0.560440,0.560440,0.672165,0.549550
Specificity,0.833333,0.875224,0.640934,0.640934,0.768402,0.912926
FalseNegRate,0.213415,0.331606,0.207254,0.207254,0.155440,0.525907
Cohen's Kappa,0.582731,0.538603,0.341243,0.341243,0.524972,0.425170
Krippendorff's Alpha,0.581198,0.538725,0.309098,0.309098,0.515676,0.421208
Phi Coef / MCC,0.588340,0.538700,0.379833,0.379833,0.549560,0.434221


As we can see from the table above, the highest precision is observed from the classifier `loc_prof_notna_and_loc_tw` at 0.656 (though at a great cost to recall). It should be noted that selecting for cases which satisfy both conditions may inadvertently select for a particular (as-yet unidentified) sub-category of user type and exclude other categories of value. The highest fscore comes from `loc_prof_notna_or_loc_tw` at 0.672, which has a precision of 0.558.

While the low scores of these metrics show that they cannot provide meaningful proxies for the manually assigned code, as classifiers they provide a suitable baseline from which to measure more sophisticated models.

## Tweet Source
An important distinguishing metadatum of a Tweet is the 'source' field, which represents the platform from which the Tweet was published. When creating a third-party application which can interact with the Twitter API, a developer must provide a descriptor string which populates this field. Because many third-party applications are designed for specific use-cases, this field provides useful information which characterises the motivations for conditions under which the Tweet was created. For example, the source `TweetMyJOBS` refers to a recruitment platform and thus is attached to Tweets advertising job listings.

We can look at the list of most common source from within the entire dataset and compare this to the sources detected during the collection period. (Note that the complete dataset will still contain a selection bias and does not necessarily characterise regular Twitter use)

In [120]:
import pandas as pd
LATEX_TABLE_FILE = 'harvey-exploration-tabs.dat'

def sanitise_tex_str(string):
    '''
    Sanitises string for use in Latex
    NOTE: Incomplete
    '''
    return string.replace('%', '\%')


def df_to_tex(df, cols=None, label='UNNAMED', caption='UNDEFINED'):
    '''
    Generates a latex table from a dataframe, 
    to be copied by user from output file to tex
    
    NOTE: format columns in the dataframe before passing to func, eg:
        df[col] = df[col].astype(int).map('{:.1f}%'.format)
        df.rename(columns={"A": "a", "B": "c"}, inplace=True)
    NOTE: linewidth and column widths are fixed and require manual adjustment
    
    input: dataframe and desired columns, 
            table reference name and caption text
    output: appends to file LATEX_TABLE_FILE
    returns: none
    '''
    df = df.copy()
    if not cols:
        cols = df.columns
    for col in [x for x in cols if x not in df.columns]:
        df[col] = ''
    df = df[cols]
    
    print('Writing df to file as tex...')
    with open(LATEX_TABLE_FILE, 'a') as a_writer:
        a_writer.write('%<*{}>'.format(label) + '\n')
#        a_writer.write('tab:' + label + '\n\n')
        a_writer.write('\\begin{table}[H]' + '\n')
        a_writer.write('  \\small\\vspace{5mm}\\centering\\extrarowsep=1mm' + '\n')
        x = 'X[-2,lm] ' * (len(df.columns)-1)
        a_writer.write('  \\begin{tabu} to .8\\linewidth { X[-1,lm] ' + x + '}' + '\n')
        a_writer.write('    \\toprule\n')
        
        row_string = '    '
        for col in cols:
            # If last column, add line ending
            if col == df.columns[-1]:
                row_string += col + ' \\\\\n'
            else:
                # Pad to length of longest column value, plus a fixed amount:
                pad = max(df[col].map(str).map(len)) + 8
                row_string += (sanitise_tex_str(col).ljust(pad) + '& ')
        a_writer.write(row_string)
        
        a_writer.write('    \\midrule\n')
        
        for index, row in df.iterrows():
            row_string = '    '
            for col in df.columns:
                if col == df.columns[-1]:
                    row_string += (sanitise_tex_str(str(row[col])) + '\\\\\n')
                else:
                    pad = max(df[col].map(str).map(len)) + 8
                    row_string += (sanitise_tex_str(str(row[col])).ljust(pad) + '& ')
            a_writer.write(row_string)
        
        a_writer.write('    \\bottomrule' + '\n')
        
        a_writer.write('  \\end{tabu}' + '\n')
        a_writer.write('  \\caption[{}]{{{}}}'.format(caption, caption) + '\n')
        a_writer.write('  \\label{{tab:{}}}'.format(label) + '\n')
        a_writer.write('\\end{table}' + '\n')
        a_writer.write('%</{}>'.format(label) + '\n\n\n')

        return

In [114]:
# View most common sources from entire dataset:
from django.db.models import Count
from streamcollect.models import Tweet

ts = Tweet.objects.all()

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

dft = pd.DataFrame.from_dict(counts[:10])
dft['Proportion'] = (dft['count'].astype(int) / ts.count()*100).map('{:.1f}%'.format)

# Title case for columns:
dft.rename(columns={x: x.title() for x in dft.columns}, inplace=True)

print('Total Tweets:', ts.count())
dft

Total Tweets: 1727438


,count,source,Proportion
0,535558,Twitter for iPhone,31.0%
1,348743,Twitter for Android,20.2%
2,342908,Twitter Web Client,19.9%
3,95360,IFTTT,5.5%
4,45830,Twitter for iPad,2.7%
5,43808,Twitter Lite,2.5%
6,42045,Instagram,2.4%
7,32273,TweetDeck,1.9%
8,28536,Facebook,1.7%
9,27105,Hootsuite,1.6%


In [119]:
# Adding descriptions for latex table, based on manual inspection and classification:
descs = [
    'first-party', 'first-party', 'first-party', 'automated process, crossposting', 'first-party',
    'first-party', 'third-party crossposting', 'first-party', 'third-party crossposting', 'social media manager'
]
dft['Description'] = pd.Series(descs)


df_to_tex(dft, cols=['Proportion', 'Source', 'Description'], label='top-tw-sources', caption='Top 10 Tweet Sources')

Writing df as tex...


In [22]:
# Count proportion of Tweets from first-party applications
first_party_sources = [
    'Twitter for iPhone',
    'Twitter for Android',
    'Twitter Web Client',
    'Twitter for iPad',
    'Twitter Lite',
    'TweetDeck',
    'Twitter for Windows',
    'Twitter for Mac',
    'Twitter for Windows Phone',
    'Twitter for BlackBerry',
    'Twitter for Android Tablets',
    'Twitter MMS'
    ]
fp_count = 0
for x in counts:
    if any([y in x['source'] for y in first_party_sources ]):
        fp_count += x['count']
print('{:.1f}% of total from first party clients: {}'.format((fp_count/ts.count()*100), fp_count))

78.5% of total from first party clients: 1355569


In [23]:
# Write to Latex variable
set_tex_var('hrvTwTotal', ts.count())
set_tex_var('hrvTwFP', fp_count)
set_tex_var('hrvTwFPPrp', fp_count/ts.count()*100, 1)

In [1]:
ts = Tweet.objects.filter(data_source=1)
print('Total for data_source=1 (keyword stream):', ts.count(), '\n')

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in counts[:20]:
    if x['count'] > 10:
        print('{:.1f}% {}: {}'.format((x['count']/ts.count()*100), x['source'], x['count']))

Total for data_source=1 (keyword stream): 31303 

28.6% Twitter for iPhone: 8959
25.5% Twitter Web Client: 7989
16.2% Twitter for Android: 5086
5.6% Paper.li: 1754
3.1% Hootsuite: 981
2.3% Instagram: 733
2.3% IFTTT: 720
2.2% Facebook: 682
2.1% TweetDeck: 658
2.1% Twitter for iPad: 652
1.9% Twitter Lite: 595
0.9% Buffer: 274
0.4% SocialNewsDesk: 138
0.4% Sprout Social: 126
0.3% Error-log: 105
0.3% Botize: 93
0.2% Periscope: 70
0.2% VoiceStorm: 69
0.2% Google: 62
0.2% Twitter for Windows: 62


In [18]:
ts = Tweet.objects.filter(data_source__gt=1, coordinates_lat__isnull=False)
print('Total for data_source=3 (geo stream)', ts.count(), '\n')

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in counts[:20]:
    if x['count'] > 10:
        print('{:.1f}% {}: {}'.format((x['count']/ts.count()*100), x['source'], x['count']))

Total for data_source=3 (geo stream) 15569 

76.6% Instagram: 11922
6.6% TweetMyJOBS: 1020
6.0% SafeTweet by TweetMyJOBS: 928
3.9% BubbleLife: 602
1.7% Foursquare: 267
1.5% Untappd: 234
0.8% Twitter for Android: 129
0.7% Hootsuite: 104
0.6% Twitter for iPhone: 87
0.4% circlepix: 55
0.3% TownTweet: 52
0.2% iOS: 28
0.2% Twitter for Android Tablets: 27
0.2% Crowdfire - Go Big: 25
0.1% Squarespace: 17
0.1% Twitter for Windows Phone: 15
0.1% Tweetbot for iΟS: 12


In [19]:
fp_count = 0
for x in counts:
    if any([y in x['source'] for y in first_party_sources ]):
        fp_count += x['count']
print('{:.1f}% of geo stream from first party clients: {}'.format((fp_count/ts.count()*100), fp_count))

NameError: name 'first_party_sources' is not defined

In [20]:
set_tex_var('hrvTwGeoTotal', ts.count())
set_tex_var('hrvTwGeoFP', fp_count)
set_tex_var('hrvTwGeoFPPrp', fp_count/ts.count()*100, 1)

val = ts.filter(source='Instagram').count()
set_tex_var('hrvTwGeoInsta', val)
set_tex_var('hrvTwGeoInstaPrp', val/ts.count()*100, 1)

val = ts.filter(source__contains='TweetMyJOBS').count()
set_tex_var('hrvTwGeoJobs', val)
set_tex_var('hrvTwGeoJobsPrp', val/ts.count()*100, 1)

Of the entire dataset of 1,727,438 Tweets, those published by first-party Twitter clients comprised 78.5% (1,355,569). In contrast, of the subset of 15,630 Tweets collected based on their location within the event's bounding box, only 1.7% (262) were published from first-party apps. Tweets crossposted by Instagram comprised 76.6% of the geotagged Tweets. The high incidence of Instagram posts in the geographic stream therefore suggest that Instagram posts are much more likely than Tweets to include geographic data, which is preserved during the crossposting process. We can check the rate across the entire dataset excluding the geo stream:

In [13]:
ts = Tweet.objects.filter(source='Instagram', data_source__lt=3)
print('Total Instagram Tweets (excl. geo stream):', ts.count())
ts_geo = ts.filter(coordinates_lat__isnull=False)
print('Total geotagged Instagram Tweets: {}, {:.1f}%'.format(ts_geo.count(), ts_geo.count()/ts.count()*100 ))

Total Instagram Tweets (excl. geo stream): 30123
Total geotagged Instagram Tweets: 9818, 32.6%


In [14]:
set_tex_var('hrvTwInstaTotalNongeo', ts.count())
set_tex_var('hrvTwInstaGeotagNongeo', ts_geo.count())
set_tex_var('hrvTwInstaGeotagNongeoPrp', ts_geo.count()/ts.count()*100, 1)

In [17]:
ts = Tweet.objects.filter(source__contains='TweetMyJOBS')
print('Total TweetMyJOBS Tweets:', ts.count())
ts_geo = ts.filter(coordinates_lat__isnull=False)
print('Total geotagged TweetMyJOBS Tweets: {}, {:.1f}%'.format(ts_geo.count(), ts_geo.count()/ts.count()*100 ))

Total TweetMyJOBS Tweets: 18517
Total geotagged TweetMyJOBS Tweets: 17150, 92.6%


In [16]:
set_tex_var('hrvTwJobsTotal', ts.count())
set_tex_var('hrvTwJobsGeotag', ts_geo.count())
set_tex_var('hrvTwJobsGeotagPrp', ts_geo.count()/ts.count()*100, 1)

As Instagram posts are traditionally based upon the publication of a recently-taken photo, the high incidence of geotagging makes this class of message highly useful in supporting the development of situational awareness.

Tweets were inspected by category (see code below) to classify the general use-case of each source and eliminate those which were only used for automated/unrelated purposes:

In [23]:
# Investigate Tweets by source to characterise source content:
# (samples are not limited to data_source>0)

ts = Tweet.objects.filter(data_source__gte=1)
fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')
source_list = [x['source'] for x in counts if x['count'] > 10]

for source in source_list:
    print('__________________\n{}:\n'.format(source))
    ts = Tweet.objects.filter(source=source)
    for t in ts.order_by('?')[:15]:
#     for t in ts.filter(data_source__gte=1).order_by('?')[:15]: # To only sample detected Tweets
        print(t.text.replace('\n', ''))

__________________
Instagram:

#hurricaneharvey #pricehike #pricegouge #texas#attorneygeneral https://t.co/sn3wnCJdiW
Photos from the Portait Mixer Professional Headshots #Headshots… https://t.co/YaryiacbC8
This is why officials have requested that citizens stay off the roads unless it essential...You… https://t.co/hyM4PaByOG
Watch all new episode of #insecure #loveandhiphophollywood #gameofthrones #allnewshit #watchnow… https://t.co/3yrYjx9iMV
Day 2#HARVEY #harveystorm #houston #flooding #weather https://t.co/uIywZs6q0M
Surprised my friends with a day of sailing since they traveled across the world to come see me… https://t.co/F9wsoAjbMY
🙏🏾 Blessings #Repost charliesangelll・・・#TheFlockas are in 25k to help #Houston we going to… https://t.co/UNjJxZBbow
Please #ShowYourLove for chrisftm_ on this #Selfie #Sunday #SS #TakeYourBestShot #TYBS #inklife… https://t.co/HJs3s6u6pD
Updated video...#HarveyWatch2017 @ The Trails at Seabourne Park, Richmond https://t.co/YiswidINOz
Just posted a phot

Join us in welcoming Matthew Blackburn, MD to the RMC Emergency Department. https://t.co/MPgfhBddzT
Penn Station repair project officially complete; Regular service resumes Sept. 5 https://t.co/RGltMwyeYi #abc7ny
#Pandilleros mataron a dos sujetos dentro de un pick up sobre la #carretera de #Oro https://t.co/vsJPJyyqvR
Be in the know for the schedule this weekend @COTA  #GoN8 #seehappy #porscheraces #porscheracing @FlyingLizard_MS @RacerInk @WCRacing https://t.co/LAyjutMsKe
RT @BostonDotCom: 200 things to do throughout Massachusetts https://t.co/W0mfeCW8NP https://t.co/bd71qJ2UMI
Mbappé al París Saint-Germain, el otro gran fichaje del siglo ► https://t.co/mJpJT2uYDf https://t.co/ziE0fa0oKa https://t.co/HnUdSSOKH2
Swingers Christa and Charles Billich talk of sex life https://t.co/sBeUhkXCpg
#TuesdayThoughts from @RepKevinBrady https://t.co/fMAEgAhN2f
#Norcorea amenazó con hundir a #EU bajo el agua https://t.co/F49jeoSB3s https://t.co/YvX1swH9V0
#Gaslighting Because why investigate your 

RT @CraigSilverman: Big conservative Facebook pages are spreading a false claim that Black Lives Matter blocked Harvey relief efforts: http…
By my estimate, over 80% of gauges in Harris County are showing water flowing out of their banks with major flooding occurring #Houstonflood https://t.co/zsJLxwjNyX
#Using #A #Dog #To #Make #A #Real #Life #Pac #Man #NashvilleStatement #PortArthur #Tiafoe #BlackLoveDoc https://t.co/kG6nubfM8P
Hey #BigCountryFB fans ... follow @ChadK3Dad for #BrownwoodLions updates tonight!
RT @DugBegley: For those thinking of renewing their registration tomorrow, @TxDMV just announced they'll be closed in HOU, BMT and Corpus
Magistrado Maikel Moreno denunció red que usó su nombre para extorsionar (+TUITS) publicado en https://t.co/VNc8qZvZnc https://t.co/xeRnfr4Gbc
#JohnMcCain likely helps #Trump #GOP &amp; #Conservatives https://t.co/je0x79RVzU #RepealAndReplace #skinny #SkinnyRepeal #SkinnyBillDead
RT @Hora20: #VocesHora20 Hinchas hicieron calle de honor para que

I'm at @Hampton Inn &amp; Suites San Antonio Northeast in San Antonio, TX https://t.co/KnWIBqN9KQ
I'm at @Burlington Coat Factory in San Antonio, TX https://t.co/r3DtoDhm6P
Chilling. (@ Irish Snug - @colfaxsnug in Denver, CO) https://t.co/1QGCIfXlCa
I'm at Parador San Antonio in Atascosa, TX https://t.co/uhqO2qbCdK
I'm at Medan in North Sumatra w/ @aliuysal1011 @asmanadia @aysearikan96 @blackegaall https://t.co/r13fJ7FkRS
Here with #Malauzai! First time doing one of these things so it should be fun! (@ MindSpark in Austin, TX) https://t.co/kMAsGvlzsq
I'm at Place Jean Jaurès in Saint-Étienne, Rhône-Alpes https://t.co/Ab5iDKtGfH
I'm at Parmer South Bistró in Austin, TX https://t.co/q5eYLVMpa7
I'm at Red Circle Ice Cream in Houston, TX https://t.co/zEuzWGvjOx https://t.co/ZuRNlU0WSe
Next stop a new country to explore! (@ Gate E11 in Houston, TX) https://t.co/jqIdIel18P
Supper Time (at @TexasRoadhouse in San Marcos, TX) https://t.co/Bkly7xx9vY
#1RoUSe #rouseband #rousehigh #fridaynightlig

"Valle Dorado"#RF_05Federer
Mira cómo los efectos visuales llevaron a #GameOfThrones a una escala épica https://t.co/q1lYC2qbT2 https://t.co/js2zfOvsLr
5 alimentos buenos para la memoria : https://t.co/kiBXZU3cAk
¡Esperen...tantito, pausa! Por una sopa de verdura, detiene su huelga de hambre… https://t.co/vf504w0zy3
Corea del Norte continúa con sus operaciones militares. 🙄https://t.co/uco4mujAVG https://t.co/voOKCu7nPM
RT pnoellozano: Un minuto de #oración por las #vocacionesOremos por los sacerdotes y religiosos.Centro_VocMty S... https://t.…
🔴🆕|| Canciller jaarreaza: Queremos y amamos al pueblo de Colombia, ellos son bienvenidos en Venezuela
Daños de huracán #Harvey en #EEUU disparan precios de combustibles, según MIC_RD. #ListínDiario #PreciosCombustiblesRD
.LulapeloBrasil visitará la ciudad más pequeña de su gira por #Brasil https://t.co/icbQlwSHPq
L’oreal fires transgender model who said ‘all white people are racist’ https://t.co/l0g8s9a0Xs
⚠️#Almomento  Corredor recibe atención d

...

Hello any one
#houstonflood #Harvey watch out for snakes https://t.co/18HfZg3bXQ
A friend wants Victoria A. to help organize a game of his favorite tabletop RPG. Is Victoria A. nerdy enough? @HSSGame
#Harvey's outer rainbands approaching central AL. Wet at times next 3 days from Harvey #alwx https://t.co/E1sVntQgXP
@realdonaldtrump is such a MAGA guy. Trump, do you think Russia, RICO, or Racism will bring you down? The 3 Rs, LOL https://t.co/hrZoyL7wq5
A Last New Orleans Neighborhood to Dry Out : NPR.    Is it going to take weeks?! For the city to drain??  https://t.co/NAbO0W2HhD
@NoelAFerguson, @AnaJuneja thanks for following. Via @FindUnfollower https://t.co/c3OeauYAl2
https://t.co/rWtWBftCFm
Volunteer mappers are helping responders reach those in need. Get involved. #hotosm #harvey https://t.co/LQeRIdxsEq
Stranded pet rescues underway amid Hurricane Harvey flooding https://t.co/cAIEaoCtMr
#Harvey Devastated My Community, But It Reminded Me How #Veterans Respond In A Time Of Need - T

Review: Answer Nineteen+ Pore Minimizer Mask ★★★☆☆ https://t.co/4VrTwroJm3 https://t.co/bQHMd9jjIC
German Foreign Minister joins calls to withdraw US nukes from the country #Germany… https://t.co/nE0bOabnCw https://t.co/brNEcaRLZa
More Flooding: TS Harvey Could Restrengthen after Return to Gulf Waters https://t.co/L9bviP3gzJ
Music: Sunday Epunam – “He Wants Your Heart” (Prod by G-Wills) https://t.co/0PzVPYT7qS https://t.co/0MtD7Rylft
Small Doctor To Expose Hacker Behind Explicit Video https://t.co/fQt5Vze0Lt https://t.co/xz5i95XaMZ
Major Houston reservoir dams overflow for first time as rain continues to dump onto Texas https://t.co/ptOdUO87ni https://t.co/XRE0SuykYH
Checkout Photos Of Dino Melaye In To Match Ferrari Car And Shirt https://t.co/4pLGLHDGpU https://t.co/XKobX2GTT8
Gist: Young Woman Goes Completely Nud3 For 20th Birthday Shoot (!8+ Photos) https://t.co/P40Xzymzv3 https://t.co/tN22w6yFRN
Nancy Pelosi Denounces Violent ‘Antifa’ Protesters: ‘Unequivocal Condemnation’ https://

Pay to play corruption https://t.co/yyaT0mJO0z By @owillis by #Shareblue
Wood Sunglasses 😎- Polarized Lenses- FREE Travel Case- Only $29.99!... https://t.co/jHAk9krwvT by #infinitiveSkies via @c0nvey https://t.co/Sn8V2WX1tU
Author @CharlesMBlow tells @MaajidNawaz he believes Trump is a "pathological liar" with... https://t.co/p0H73oc5vw by #LBC via @c0nvey https://t.co/lpHKAWc2xC
No new #TeenWolf until Sept. 3rd, but you can re-watch the craziness from last... https://t.co/Y9ZEo2LzGB by #MTVteenwolf via @c0nvey https://t.co/IVvExbDNUs
"Devastating wildfires or the return of Phaethon" by Pablo Rodas-Martini https://t.co/nBlXJx9gxB #climate... by #pablorodas via @c0nvey https://t.co/mTAWMztA2r
Astonishing. Bots produce 84% of #Russian tweets about #NATO in eastern Europe. And... https://t.co/kt9G2F1O2T by #KvaakFi via @c0nvey
Well done to everyone involved in the campaign. Labour win by-election seats in North... https://t.co/EnAvy7FE4s by #PeoplesMomentum
It could have all been so diffe

@inderarmour #TTOTT #TsuMan #jnj @donateaphoto @SavetheChildren https://t.co/airwkMI0QC https://t.co/PHBz9ZbVAH
Galveston #jnj @donateaphoto #HurricaneHarvey https://t.co/XQNf8OKqkf https://t.co/eqeZowS7wY
Wish Rainfall passes Houston soon. Everyone be safe! #jnj @donateaphoto https://t.co/DOkoy7jD46 https://t.co/9VRjnns4dd
#jnj @donateaphoto #HurricaneHarvey https://t.co/YWdJImpPcl https://t.co/Oji5FumMVo
From the @westinbonaventure looking at @dodgers lights in the distance. #ilovela#jnj @donateaphoto #DoGoodGiveGoods https://t.co/JjbdEE1nZd https://t.co/Kb6fyQDaNt
Getting a jump start on my day... #DonateAPhoto  #jnj @donateaphoto #UntilWeHeal https://t.co/1O5C6hRkI0 https://t.co/M6dx5Ye00U
Everyday is a blessing. #donateaphoto #jnj @donateaphoto #HurricaneHarvey https://t.co/3v7oxT3M0o https://t.co/QhmHnV15Xt
#jnj #donateaphoto https://t.co/HkZDv2rlHC https://t.co/8dp0r3HPvj
#jnj @donateaphoto #HurricaneHarvey#charity #help #texas  you want to help the people in texas get this app 

You guys, there's a #maskshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #backdropshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #dockshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #radiosondeshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #softballshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #producershortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #walnutshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #parentshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #beltshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #mochashortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's an #aldershortage in Texas! #BREAKING #Harvey
You guys, there's a #handshortage in Texas! #BREAKING #Harvey #GasShortage
You guys, there's a #difficultyshortage in Texas! #BREAKING #Harvey #GasShortage
You guy

* Instagram -- Crossposting images
* Twitter for iPhone
* Twitter Web Client
* Twitter for Android
* Paper.li -- Automated, seems like spam from a single source
* Hootsuite -- Suite, but appears to be used by 'regular users'
* TweetMyJOBS -- Automated job listings
* SafeTweet by TweetMyJOBS -- Automated job listings
* IFTTT -- Appears to be commonly used for cross-posting from eg Instagram
* Facebook
* TweetDeck
* Twitter for iPad
* BubbleLife -- Suite, news
* Twitter Lite
* Buffer -- Suite, news
* Foursquare -- Crossposting check-in app, sometimes informative, but mostly "I'm at x" template
* Untappd -- Crossposting check-in app (for beer drinkers), rarely informative
* SocialNewsDesk -- Suite, news
* Sprout Social -- Suite, news
* Error-log -- Automated bot / spam
* Botize -- All in Spanish, appeared to be only news
* Tweetbot for iΟS -- third-party app, regular users
* Periscope -- Crossposting, live video feeds
* VoiceStorm -- Suite, CRM
* Twitter for Windows
* Google -- Crossposting, mostly from Youtube (user likes or posts a video)
* Hearsay Social -- Suite, news
* despa ringtones -- Automated bot / spam
* circlepix -- Automated retail listings
* TweetCaster for Android -- Third party app
* TownTweet -- Automated forecasts
* Twitter for Mac
* Mobile Web (M2) -- (mobile.twitter.com)
* HubSpot -- Suite, CRM
* Crowdfire - Go Big -- Suite, news
* iOS
* Sprinklr -- Suite, CRM
* LinkedIn -- News article links
* Echofon -- Third-party app, regular users
* SeeYourWeather.com Galveston -- Automated weather bot
* WordPress.com -- Automated blog reposting
* GaggleAMP -- Suite, recruitment
* Twitter for Windows Phone
* Twitter for Android Tablets
* Radian6 -Social Media Management -- Automated news
* Convey: Make it post for you -- Automated news
* Cloudhopper -- Appears to be a Suite, CRM
* dlvr.it -- Automated news
* Twitter for BlackBerry
* Squarespace -- Automated blog reposting
* SocialFlow -- Suite, news
* Donate a Photo -- Looks like some J&J sponsored app
* Twitterrific -- Third-party app, regular users
* a7aa7o -- Automated bot / spam
* CoSchedule -- Suite, CRM
* despacitoapp -- Automated bot / spam
* shortage bot -- Automated bot / spam
* Bambu by Sprout Social -- Suite, CRM
* SocialPilot.co -- Suite, CRM
* Weather Message -- Automated weather reports
* RadarScope -- Weather enthusiast app


Therefore a list of sources can be created which are not used by 'regular users':

We can now compare the rate of useful messages for each source categorisation:

In [24]:
fp = ['Twitter for iPhone',
 'Twitter Web Client',
 'Twitter for Android',
 'TweetDeck',
 'Twitter for iPad',
 'Twitter Lite',
 'Twitter for Windows',
 'Twitter for Mac',
 'Mobile Web (M2)',
 'iOS',
 'Twitter for Windows Phone',
 'Twitter for Android Tablets',
 'Twitter for BlackBerry']

tp = [
 'Tweetbot for iΟS',
 'TweetCaster for Android',
 'Echofon',
 'Twitterrific']
      
cross = ['Instagram',
 'IFTTT',
 'Facebook',
 'Foursquare',
 'Untappd',
 'Periscope',
 'Google',
 'LinkedIn',
 'WordPress.com',
 'Squarespace',
 'RadarScope']

cross2 = ['IFTTT',
 'Facebook',
 'Foursquare',
 'Untappd',
 'Periscope',
 'Google',
 'LinkedIn',
 'WordPress.com',
 'Squarespace',
 'RadarScope']
      
suites = [
 'Hootsuite',
 'BubbleLife',
 'Buffer',
 'SocialNewsDesk',
 'Sprout Social',
 'Botize',
 'VoiceStorm',
 'Hearsay Social',
 'HubSpot',
 'Crowdfire - Go Big',
 'Sprinklr',
 'GaggleAMP',
 'Radian6 -Social Media Management',
 'Convey: Make it post for you',
 'Cloudhopper',
 'SocialFlow',
 'CoSchedule',
 'Bambu by Sprout Social',
 'SocialPilot.co']
      
auto = [
 'TweetMyJOBS',
 'SafeTweet by TweetMyJOBS',
 'circlepix',
 'TownTweet',
 'SeeYourWeather.com Galveston',
 'dlvr.it',
 'Donate a Photo',
 'a7aa7o',
 'Weather Message']
      
spam = ['Paper.li',
 'Error-log',
 'despa ringtones',
 'shortage bot',
 'despacitoapp']

source_cats = {'First Party': fp, 'Third Party': tp, 'Crossposting': cross,
               'Crossposting excl. Insta': cross2, 'Suites': suites, 
               'Automated': auto, 'Spam': spam}

tweet_codings = (Coding.objects
                    .filter(coding_id=1)
                    .filter(tweet__isnull=False)
                    .filter(data_code__data_code_id__gt=0)
                  )
ts = Tweet.objects.filter(coding_for_tweet__in=tweet_codings)
tw_df = pd.DataFrame(list(ts.values('tweet_id', 'source', 'data_source')))

desired_codes = ['Aid Request', 'Ground Truth']
#TODO: check why get returned two vals for some here? :
tw_df['is_desired'] = tw_df['tweet_id'].apply(lambda x: True if tweet_codings.filter(tweet__tweet_id=x)[0].data_code.name in desired_codes else False)
desired_series = tw_df['is_desired']

for cat in source_cats:
    for source in source_cats[cat]:
        s1 = tw_df['source'].apply(lambda x: True if x == source else False)
        pos = sum(s1 & desired_series)
        tot = sum(s1)
        if tot == 0:
            continue
        conf = confusion_matrix_from_series(s1, desired_series)
        agr = calc_agreement_metrics(conf)
        prop = pos/tot*100
        print('{:.1f}% -- {}: {} of {} -- Pr: {:.3f}, R: {:.3f}'.format(
            prop, source, pos, tot, agr['Prec'], agr['Recall']
            ))

    s1 = tw_df['source'].apply(lambda x: True if x in source_cats[cat] else False)
    conf = confusion_matrix_from_series(s1, desired_series)
    agr = calc_agreement_metrics(conf)
    pos = sum(s1 & desired_series)
    tot = sum(s1)
    
    
    print('{}: {:.2f}% of {}'.format(cat, pos/tot*100, tot))
    print('Precision: {:.3f}, Recall: {:.3f}\n'.format(agr['Prec'], agr['Recall']))

14.6% -- Twitter for iPhone: 59 of 405 -- Pr: 0.146, R: 0.144
8.0% -- Twitter Web Client: 33 of 415 -- Pr: 0.080, R: 0.081
13.0% -- Twitter for Android: 31 of 238 -- Pr: 0.130, R: 0.076
8.6% -- TweetDeck: 3 of 35 -- Pr: 0.086, R: 0.007
5.7% -- Twitter for iPad: 2 of 35 -- Pr: 0.057, R: 0.005
10.0% -- Twitter Lite: 3 of 30 -- Pr: 0.100, R: 0.007
0.0% -- Twitter for Windows: 0 of 7 -- Pr: 0.000, R: 0.000
0.0% -- Twitter for Mac: 0 of 2 -- Pr: 0.000, R: 0.000
0.0% -- Mobile Web (M2): 0 of 5 -- Pr: 0.000, R: 0.000
66.7% -- Twitter for Windows Phone: 2 of 3 -- Pr: 0.667, R: 0.005
0.0% -- Twitter for BlackBerry: 0 of 2 -- Pr: 0.000, R: 0.000
First Party: 11.30% of 1177
Precision: 0.113, Recall: 0.325

0.0% -- Tweetbot for iΟS: 0 of 1 -- Pr: 0.000, R: 0.000
0.0% -- TweetCaster for Android: 0 of 5 -- Pr: 0.000, R: 0.000
Third Party: 0.00% of 6
Precision: 0.000, Recall: 0.000

33.7% -- Instagram: 247 of 733 -- Pr: 0.337, R: 0.604
17.6% -- IFTTT: 6 of 34 -- Pr: 0.176, R: 0.015
17.1% -- Facebook:

/home/rosles/projects/crisis-data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars


75.0% -- Periscope: 3 of 4 -- Pr: 0.750, R: 0.007
25.0% -- Google: 1 of 4 -- Pr: 0.250, R: 0.002
0.0% -- LinkedIn: 0 of 2 -- Pr: 0.000, R: 0.000
0.0% -- WordPress.com: 0 of 2 -- Pr: 0.000, R: 0.000
Crossposting excl. Insta: 17.48% of 103
Precision: 0.175, Recall: 0.044

10.0% -- Hootsuite: 6 of 60 -- Pr: 0.100, R: 0.015
1.8% -- BubbleLife: 1 of 57 -- Pr: 0.018, R: 0.002
0.0% -- Buffer: 0 of 14 -- Pr: 0.000, R: 0.000
9.1% -- SocialNewsDesk: 1 of 11 -- Pr: 0.091, R: 0.002
0.0% -- Sprout Social: 0 of 3 -- Pr: 0.000, R: 0.000
0.0% -- Botize: 0 of 6 -- Pr: 0.000, R: 0.000
100.0% -- VoiceStorm: 1 of 1 -- Pr: 1.000, R: 0.002
0.0% -- Hearsay Social: 0 of 3 -- Pr: 0.000, R: 0.000
0.0% -- HubSpot: 0 of 1 -- Pr: 0.000, R: 0.000
25.0% -- Crowdfire - Go Big: 1 of 4 -- Pr: 0.250, R: 0.002
0.0% -- GaggleAMP: 0 of 2 -- Pr: 0.000, R: 0.000
0.0% -- Radian6 -Social Media Management: 0 of 1 -- Pr: 0.000, R: 0.000
0.0% -- Cloudhopper: 0 of 2 -- Pr: 0.000, R: 0.000
Suites: 6.06% of 165
Precision: 0.061, Rec

In [25]:
# Re-Printing output for Latex table

source_cats = {'First Party': fp, 'Third Party': tp, 'Crossposting': cross, 
               'Suites': suites, 'Automated': auto, 'Spam': spam}

for cat in source_cats:
    for source in source_cats[cat]:
        s1 = tw_df['source'].apply(lambda x: True if x == source else False)
        tot = sum(s1)
        if tot <= 9:
            continue
        conf = confusion_matrix_from_series(s1, desired_series)
        agr = calc_agreement_metrics(conf)
        prop = pos/tot*100
        print('{} & {} & {:.3f} & {:.3f} \\\\'.format(source, tot, agr['Prec'], agr['Recall']))

print('\n')

for cat in source_cats:
    s1 = tw_df['source'].apply(lambda x: True if x in source_cats[cat] else False)
    conf = confusion_matrix_from_series(s1, desired_series)
    agr = calc_agreement_metrics(conf)
    tot = sum(s1)
    print('{} & {} & {:.2f} & {:.2f} \\\\'.format(cat, tot, agr['Prec'], agr['Recall']))

Twitter for iPhone & 405 & 0.146 & 0.144 \\
Twitter Web Client & 415 & 0.080 & 0.081 \\
Twitter for Android & 238 & 0.130 & 0.076 \\
TweetDeck & 35 & 0.086 & 0.007 \\
Twitter for iPad & 35 & 0.057 & 0.005 \\
Twitter Lite & 30 & 0.100 & 0.007 \\
Instagram & 733 & 0.337 & 0.604 \\
IFTTT & 34 & 0.176 & 0.015 \\
Facebook & 35 & 0.171 & 0.015 \\
Untappd & 13 & 0.000 & 0.000 \\
Hootsuite & 60 & 0.100 & 0.015 \\
BubbleLife & 57 & 0.018 & 0.002 \\
Buffer & 14 & 0.000 & 0.000 \\
SocialNewsDesk & 11 & 0.091 & 0.002 \\
TweetMyJOBS & 90 & 0.000 & 0.000 \\
SafeTweet by TweetMyJOBS & 101 & 0.000 & 0.000 \\
Paper.li & 74 & 0.000 & 0.000 \\


First Party & 1177 & 0.11 & 0.33 \\
Third Party & 6 & 0.00 & 0.00 \\
Crossposting & 836 & 0.32 & 0.65 \\
Suites & 165 & 0.06 & 0.02 \\
Automated & 204 & 0.00 & 0.00 \\
Spam & 87 & 0.00 & 0.00 \\


/home/rosles/projects/crisis-data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars


The sources classified as spam, automated and suites (once Hootsuite is excluded) have neglible or zero rates of useful content and may therefore be eliminated from the dataset. Crossposting apps vary by app and should be classified individually. The sample of third-party apps is too small to draw conclusions.

The list of unwanted sources is then defined:

In [26]:
unwanted_sources = [
    'Paper.li',
    'TweetMyJOBS',
    'SafeTweet by TweetMyJOBS',
    'BubbleLife',
    'Buffer',
#     'Foursquare', ###
    'Untappd', ###
    'SocialNewsDesk',
    'Sprout Social',
    'Error-log',
    'Botize',
    'VoiceStorm',
    'Google', ###
    'Hearsay Social',
    'despa ringtones',
    'circlepix',
    'TownTweet',
    'HubSpot',
    'Crowdfire - Go Big',
    'Sprinklr',
    'LinkedIn',
    'SeeYourWeather.com Galveston',
    'WordPress.com',
    'GaggleAMP',
    'Radian6 -Social Media Management',
    'Convey: Make it post for you',
    'Cloudhopper',
    'dlvr.it',
    'Squarespace',
    'SocialFlow',
    'Donate a Photo',
    'a7aa7o',
    'CoSchedule',
    'despacitoapp',
    'shortage bot',
    'Bambu by Sprout Social',
    'SocialPilot.co',
    'Weather Message',
    'RadarScope']

To verify these classifications, we can compare the content of each source subset using the coded Tweet dataset. Where a significant proportion of the subset has been classified as a 'desired code', the decision to eliminate the source should be reconsidered.

In [27]:
# #from django.db.models import Count
# tweet_codings = (Coding.objects
#                     .filter(coding_id=1)
#                     .filter(tweet__isnull=False)
#                     .filter(data_code__data_code_id__gt=0)
#                   )

# #desired_codes = ['Aid Request', 'Ground Truth', 'Info for Affected', 'Emotion - Affected']
# desired_codes = ['Aid Request', 'Ground Truth']
to_inspect = []

for source in unwanted_sources:
    cs = tweet_codings.filter(tweet__source=source)
    tot = cs.count()
    if tot == 0:
        continue
    fieldname = 'data_code__name'
    counts = cs.values(fieldname).annotate(the_count=Count(fieldname))
    n = 0
    for c in counts:
        if c['data_code__name'] in desired_codes:
            n += c['the_count']
    if tot >= 1:
        print('{:.1f}% -- {}: {} of {}'.format(n/tot*100, source, n, tot))
        # If more than 10% desired, add to list for manual inspection:
        if n/tot > .1:
            to_inspect.append(source) 

0.0% -- Paper.li: 0 of 74
0.0% -- TweetMyJOBS: 0 of 90
0.0% -- SafeTweet by TweetMyJOBS: 0 of 101
1.8% -- BubbleLife: 1 of 57
0.0% -- Buffer: 0 of 14
0.0% -- Untappd: 0 of 13
9.1% -- SocialNewsDesk: 1 of 11
0.0% -- Sprout Social: 0 of 3
0.0% -- Error-log: 0 of 8
0.0% -- Botize: 0 of 6
100.0% -- VoiceStorm: 1 of 1
25.0% -- Google: 1 of 4
0.0% -- Hearsay Social: 0 of 3
0.0% -- despa ringtones: 0 of 5
0.0% -- circlepix: 0 of 2
0.0% -- TownTweet: 0 of 8
0.0% -- HubSpot: 0 of 1
25.0% -- Crowdfire - Go Big: 1 of 4
0.0% -- LinkedIn: 0 of 2
0.0% -- SeeYourWeather.com Galveston: 0 of 1
0.0% -- WordPress.com: 0 of 2
0.0% -- GaggleAMP: 0 of 2
0.0% -- Radian6 -Social Media Management: 0 of 1
0.0% -- Cloudhopper: 0 of 2
0.0% -- Donate a Photo: 0 of 1
0.0% -- Weather Message: 0 of 1


In [28]:
# Manually inspect the Tweets from the sources with >10% useful codes:

for source in to_inspect:
    cs = tweet_codings.filter(tweet__source=source)
    cs2 = cs.filter(data_code__name__in=desired_codes)
    print('\n{}: {} of {}'.format(source, cs2.count(), cs.count()))
    for c in cs2:
        print(c.tweet.text)


VoiceStorm: 1 of 1
#Harvey is strong, but hearts are stronger. https://t.co/te1uSYYtdp https://t.co/DgMCKaWwvN

Google: 1 of 4
I added a video to a @YouTube playlist https://t.co/cnFgr5WDZa Aerial view: Texas Flooding Aftermath [part 3] #Houston #TXwx #Harvey

Crowdfire - Go Big: 1 of 4
MỚI NHẤT** HOUSTON** , TEXAS** CHÌM TRONG BIỂN NƯỚC SAU SIÊU BÃO **HARVEY** - YouTube
https://t.co/GlnCiqwIV8 https://t.co/JSZgmmxZEK


The qualitative inspection of the sources with a significant proportion of useful information shows that the positively coded Tweets are not strongly associated with their classification, and as single instances, are only flagged due to the low sample size. Therefore these discrepencies may be ignored.

Now we can check how much of the dataset was made up of the uninformative sources and thus measure the impact of early elimination of these data:

In [29]:
ts = Tweet.objects.filter(data_source__gt=1)
print('Total for data_source=3 (geo stream)', ts.count())
ts2 = ts.exclude(source__in=unwanted_sources)
print('Total from accepted sources', ts2.count())
print('Total removed: {} ({:.2f}%)\n'.format((ts.count() - ts2.count()), 
                                           ((ts.count() - ts2.count())/ts.count())*100))

print('Accepted-source adjusted source distribution:')
fieldname = 'source'
counts = ts2.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in [c for c in counts if c['count'] > 10]:
    print('{:.1f}% {}: {}'.format((x['count']/ts2.count()*100), x['source'], x['count']))

Total for data_source=3 (geo stream) 15569
Total from accepted sources 12628
Total removed: 2941 (18.89%)

Accepted-source adjusted source distribution:
94.4% Instagram: 11922
2.1% Foursquare: 267
1.0% Twitter for Android: 129
0.8% Hootsuite: 104
0.7% Twitter for iPhone: 87
0.2% iOS: 28
0.2% Twitter for Android Tablets: 27
0.1% Twitter for Windows Phone: 15
0.1% Tweetbot for iΟS: 12


In [30]:
ts = Tweet.objects.filter(data_source=1)
print('Total for data_source=1 (keyword stream)', ts.count())
ts2 = ts.exclude(source__in=unwanted_sources)
print('Total from accepted sources', ts2.count())
print('Total removed: {} ({:.2f}%)\n'.format((ts.count() - ts2.count()), 
                                           ((ts.count() - ts2.count())/ts.count())*100))

print('Accepted-source adjusted source distribution:')
fieldname = 'source'
counts = ts2.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in [c for c in counts if c['count'] > 10]:
    print('{:.1f}% {}: {}'.format((x['count']/ts2.count()*100), x['source'], x['count']))

Total for data_source=1 (keyword stream) 31303
Total from accepted sources 28081
Total removed: 3222 (10.29%)

Accepted-source adjusted source distribution:
31.9% Twitter for iPhone: 8959
28.4% Twitter Web Client: 7989
18.1% Twitter for Android: 5086
3.5% Hootsuite: 981
2.6% Instagram: 733
2.6% IFTTT: 720
2.4% Facebook: 682
2.3% TweetDeck: 658
2.3% Twitter for iPad: 652
2.1% Twitter Lite: 595
0.2% Periscope: 70
0.2% Twitter for Windows: 62
0.2% Tweetbot for iΟS: 59
0.2% Twitter for Mac: 51
0.2% TweetCaster for Android: 51
0.2% Mobile Web (M2): 50
0.1% Echofon: 40
0.1% Twitter for BlackBerry: 19
0.1% Twitter for Windows Phone: 16
0.1% iOS: 15
0.0% Twitterrific: 14


We can test the impact of removing the 'undesired' sources on the coded dataset, which shows improvements in 'precision' of the subsets at a very small cost to recall (2-3 relevant messages dropped).

In [31]:
for i in [1,3]:
    print('\nData Source:', i)
    a = tw_df.loc[tw_df['data_source'] == i]['is_desired']
    b = tw_df.loc[(tw_df['data_source'] == i) & (~tw_df['source'].isin(unwanted_sources))]['is_desired']
    print(a.value_counts(), a.shape, sum(a.value_counts()))
    print(b.value_counts(), b.shape)

    print('Original precision: {:.3f}, Filtered precision: {:.3f}'.format(a.value_counts()[1]/sum(a.value_counts()), 
                                                                b.value_counts()[1]/sum(b.value_counts()) ))
    print('Original recall: {:.3f}, Filtered recall: {:.3f}'.format(a.value_counts()[1]/a.value_counts()[1], 
                                                                b.value_counts()[1]/a.value_counts()[1] ))
    
print('\nCombined')
a = tw_df.loc[tw_df['data_source'] > 0]['is_desired']
b = tw_df.loc[(tw_df['data_source'] > 0) & (~tw_df['source'].isin(unwanted_sources))]['is_desired']
print(a.value_counts(), a.shape, sum(a.value_counts()))
print(b.value_counts(), b.shape)

print('Original precision: {:.3f}, Filtered precision: {:.3f}'.format(a.value_counts()[1]/sum(a.value_counts()), 
                                                            b.value_counts()[1]/sum(b.value_counts()) ))
print('Original recall: {:.3f}, Filtered recall: {:.3f}'.format(a.value_counts()[1]/a.value_counts()[1], 
                                                            b.value_counts()[1]/a.value_counts()[1] ))


Data Source: 1
False    1343
True      166
Name: is_desired, dtype: int64 (1509,) 1509
False    1201
True      163
Name: is_desired, dtype: int64 (1364,)
Original precision: 0.110, Filtered precision: 0.120
Original recall: 1.000, Filtered recall: 0.982

Data Source: 3
False    748
True     243
Name: is_desired, dtype: int64 (991,) 991
False    478
True     241
Name: is_desired, dtype: int64 (719,)
Original precision: 0.245, Filtered precision: 0.335
Original recall: 1.000, Filtered recall: 0.992

Combined
False    2091
True      409
Name: is_desired, dtype: int64 (2500,) 2500
False    1679
True      404
Name: is_desired, dtype: int64 (2083,)
Original precision: 0.164, Filtered precision: 0.194
Original recall: 1.000, Filtered recall: 0.988
